# Kaggle Competition: Titanic - Machine Learning from Disaster
# Machine Learning with scikit-learn

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

## Prepare datasets

In [2]:
# train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
# test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_df.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train_df.set_index('PassengerId', inplace=True)
test_df.set_index('PassengerId', inplace=True)

# Drop unnecessary columns
train_df = train_df.drop(columns=['Name', 'Cabin', 'Ticket'])
test_df = test_df.drop(columns=['Name', 'Cabin', 'Ticket'])

In [6]:
# Check missing values
print('train_df:\n')
print(train_df.isnull().sum())

print('\ntest_df:\n')
print(test_df.isnull().sum())

train_df:

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

test_df:

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64


In [7]:
# Fill missing values
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

In [8]:
# Change Age to a categorical variable
age_bins = [0, 15, 35, 55, float('inf')]
age_labels = ['child', 'young', 'adult', 'elder']

train_df['Age_Category'] = pd.cut(train_df['Age'], bins=age_bins, labels=age_labels, right=False)
test_df['Age_Category'] = pd.cut(test_df['Age'], bins=age_bins, labels=age_labels, right=False)

train_df.drop('Age', axis=1, inplace=True)
test_df.drop('Age', axis=1, inplace=True)

In [9]:
train_df = pd.get_dummies(train_df, columns=['Sex', 'Embarked', 'Age_Category'])
test_df = pd.get_dummies(test_df, columns=['Sex', 'Embarked', 'Age_Category'])

In [10]:
train_df.head(5)

,Survived,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age_Category_child,Age_Category_young,Age_Category_adult,Age_Category_elder
PassengerId,,,,,,,,,,,,,,
1,0,3,1,0,7.2500,False,True,False,False,True,False,True,False,False
2,1,1,1,0,71.2833,True,False,True,False,False,False,False,True,False
3,1,3,0,0,7.9250,True,False,False,False,True,False,True,False,False
4,1,1,1,0,53.1000,True,False,False,False,True,False,False,True,False
5,0,3,0,0,8.0500,False,True,False,False,True,False,False,True,False


In [11]:
test_df.head(5)

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age_Category_child,Age_Category_young,Age_Category_adult,Age_Category_elder
PassengerId,,,,,,,,,,,,,
892,3,0,0,7.8292,False,True,False,True,False,False,True,False,False
893,3,1,0,7.0000,True,False,False,False,True,False,False,True,False
894,2,0,0,9.6875,False,True,False,True,False,False,False,False,True
895,3,0,0,8.6625,False,True,False,False,True,False,True,False,False
896,3,1,1,12.2875,True,False,False,False,True,False,True,False,False


In [12]:
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Model Building

### Logistic Regression

In [13]:
lr_params = {'C': [1, 10, 100, 1000]}
lr = LogisticRegression(max_iter=1000, penalty='l2', random_state=0)
lr_gs = GridSearchCV(estimator=lr, param_grid=lr_params, cv=5, scoring='accuracy')

lr_gs.fit(X_train, y_train)
prediction_lr = lr_gs.predict(X_test)
acu = accuracy_score(prediction_lr, y_test)

print('Logistic regression best params: {}'.format(lr_gs.best_params_))
print('Logistic regression accuracy: {}'.format(acu))

Logistic regression best params: {'C': 100}
Logistic regression accuracy: 0.8100558659217877


### Support Vector Machine

In [14]:
svm_params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [1, 10, 100]
}
svm = SVC(random_state=0)
svm_gs = GridSearchCV(estimator=svm, param_grid=svm_params, cv=5, scoring='accuracy')

svm_gs.fit(X_train, y_train)
prediction_svm = svm_gs.predict(X_test)
accu = accuracy_score(prediction_svm, y_test)

print('SVM best params: {}'.format(svm_gs.best_params_))
print('SVM accuracy: {}'.format(accu))

SVM best params: {'C': 100, 'kernel': 'linear'}
SVM accuracy: 0.7988826815642458


### Decision Tree

In [15]:
tree_params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [3, 4, 5, 7, 10, 15],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [2, 3, 4, 5, 6]
}

tree = DecisionTreeClassifier(random_state=0)
tree_gs = GridSearchCV(estimator=tree, param_grid=tree_params, cv=5, scoring='accuracy')

tree_gs.fit(X_train, y_train)
prediction_tree = tree_gs.predict(X_test)
acu = accuracy_score(prediction_tree, y_test)

print('Decision tree best params: {}'.format(tree_gs.best_params_))
print('Decision tree accuracy: {}'.format(acu))

Decision tree best params: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
Decision tree accuracy: 0.8268156424581006


### Random Forest

In [16]:
forest_params = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [3, 4, 5],
    'min_samples_leaf': [2, 3, 5, 7],
    'min_samples_split': [5, 6, 7, 10]
}

forest = RandomForestClassifier(n_jobs=-1, random_state=0)
forest_gs = GridSearchCV(estimator=forest, param_grid=forest_params, cv=5, scoring='accuracy')

forest_gs.fit(X_train, y_train)
prediction_forest = forest_gs.predict(X_test)
acu = accuracy_score(prediction_forest, y_test)

print('Random forest best params: {}'.format(forest_gs.best_params_))
print('Random forest accuracy: {}'.format(acu))

Random forest best params: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 7, 'n_estimators': 50}
Random forest accuracy: 0.8268156424581006


### Voting Classifier

In [17]:
# Prepare the previous classifiers with the best parameters
vt_lr = LogisticRegression(**lr_gs.best_params_, max_iter=1000, penalty='l2', random_state=0)
vt_svm = SVC(**svm_gs.best_params_, probability=True, random_state=0)
vt_tree = DecisionTreeClassifier(**tree_gs.best_params_, random_state=0)
vt_forest = RandomForestClassifier(**forest_gs.best_params_, random_state=0)

# Build voting classifier
vt = VotingClassifier(estimators=[
    ('vt_lr', vt_lr),
    ('vt_svm', vt_svm),
    ('vt_tree', vt_tree),
    ('vt_forest', vt_forest)
], voting='hard')

vt.fit(X_train, y_train)
prediction_vt = vt.predict(X_test)
acu = accuracy_score(prediction_vt, y_test)

print('Voting classifier accuracy: {}'.format(acu))

Voting classifier accuracy: 0.8212290502793296


In [18]:
# Make final prediction and prepare submission
submission = pd.DataFrame({
    'PassengerId': test_df.index,
    'Survived': vt.predict(test_df)
})
submission.to_csv('submission.csv', index=False)